In [2]:
import pandas as pd
from pathlib import Path


In [ ]:
Path = ("/Users/CAM/Desktop/DBMF/data/HistoricalData_1664150906387.csv")

In [18]:
dbmf_df = pd.read_csv("/Users/CAM/Desktop/DBMF/data/HistoricalData_1664150906387.csv", index_col = "Date")

In [19]:
dbmf_df

,Close/Last,Volume,Open,High,Low
Date,,,,,
09/23/2022,33.8400,1335332.0,33.6900,33.9500,33.6000
09/22/2022,33.4700,996129.0,33.3300,33.6100,33.2508
09/21/2022,33.4000,1030096.0,33.2700,33.6600,33.1000
09/20/2022,33.2200,877852.0,33.1200,33.4900,33.1200
09/19/2022,32.9900,445331.0,32.9800,33.2887,32.6700
...,...,...,...,...,...
05/14/2019,24.9824,216.0,24.9600,25.0000,24.9600
05/13/2019,24.8395,20.0,24.8395,24.8395,24.8395
05/10/2019,25.0226,109.0,25.0226,25.0226,25.0226


In [20]:
dbmf_df.describe()

,Close/Last,Volume,Open,High,Low
count,853.000000,8.260000e+02,853.000000,853.000000,853.000000
mean,27.376437,6.037820e+04,27.389239,27.533638,27.262367
std,2.246819,1.686679e+05,2.248498,2.305711,2.196013
min,24.017300,1.000000e+00,24.000000,24.017300,24.000000
25%,25.615000,8.762500e+02,25.634200,25.710000,25.546900
50%,27.211500,4.427000e+03,27.240000,27.280000,27.157200
75%,28.380000,2.370175e+04,28.410000,28.690000,28.223000
max,33.840000,1.659054e+06,33.690000,33.950000,33.600000
